In [57]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/joeday/Documents/Fantasy/fantasy_football_season_projections')

from config import INPUT_DATA_LOC, OUTPUT_DATA_LOC, MIN_YR, MAX_YR, NEEDED_COLUMNS, MAX_WEEK

In [58]:
game_week_map = pd.read_csv(OUTPUT_DATA_LOC + 'game_week_map.csv')
fantasy_plays = pd.read_csv(OUTPUT_DATA_LOC + 'fantasy_plays.csv')
weeks_played = pd.read_csv(OUTPUT_DATA_LOC + 'weeks_played.csv')
player_id_map = pd.read_csv(OUTPUT_DATA_LOC + 'player_id_map.csv')
position_map = pd.read_csv(OUTPUT_DATA_LOC + 'position_map.csv')
roster_map = pd.read_csv(OUTPUT_DATA_LOC + 'roster_map.csv')

/Users/joeday/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Team Rushing Stats

### Rushes per Team by Season

In [59]:
rushing_plays = fantasy_plays[fantasy_plays['play_type'] == 'run']
total_rushes_by_team = rushing_plays.groupby(['posteam', 'season'])['play_id'].count().reset_index()
total_rushes_by_team.columns = ['posteam', 'season', 'team_carries']
total_rushes_by_team['team_carries_per_game'] = total_rushes_by_team['team_carries'] / (MAX_WEEK-1)

In [60]:
total_rushes_by_team

,posteam,season,team_carries,team_carries_per_game
0,ARI,2009,331,22.066667
1,ARI,2010,300,20.000000
2,ARI,2011,349,23.266667
3,ARI,2012,332,22.133333
4,ARI,2013,382,25.466667
...,...,...,...,...
348,WAS,2015,382,25.466667
349,WAS,2016,353,23.533333
350,WAS,2017,376,25.066667
351,WAS,2018,395,26.333333


### Carries per Game per Player by Season

In [61]:
rushing_plays = fantasy_plays[fantasy_plays['play_type'] == 'run']

carries_by_player = rushing_plays.\
                                groupby(['rusher_player_id',
                                         'season'])\
                                ['play_id'].\
                                count().\
                                reset_index()

carries_by_player.columns = ['rusher_player_id',
                             'season',
                             'carries']

carries_by_player = carries_by_player.sort_values(by='rusher_player_id')

rushing_plays['capped_rushing_yds'] = np.where(rushing_plays['yards_gained'] > 15,
                                               15,
                                               rushing_plays['yards_gained'])

rushing_yds_by_player = rushing_plays.\
                                groupby(['rusher_player_id',
                                         'season'])\
                                [['yards_gained', 'capped_rushing_yds', 'rush_touchdown', 'fumble']].\
                                sum().\
                                reset_index()

rushing_df = carries_by_player.merge(rushing_yds_by_player,
                             how='inner',
                             on=['rusher_player_id','season']
                            )

In [62]:
rushing_df = rushing_df.\
                merge(weeks_played,
                      how='inner',
                      left_on=['rusher_player_id','season'],
                      right_on=['player_id','season'])

rushing_df.drop('player_id', axis=1, inplace=True)

rushing_df.drop_duplicates(subset=['rusher_player_id','season'], keep='first', inplace=True)

In [63]:
rushing_df['carries_per_game'] = \
    rushing_df['carries'] / \
    rushing_df['games_played']

rushing_df['yards_per_game'] = \
    rushing_df['yards_gained'] / \
    rushing_df['games_played']

rushing_df['yards_per_carry'] = \
    rushing_df['yards_gained'] / \
    rushing_df['carries']

rushing_df['capped_yards_per_game'] = \
    rushing_df['capped_rushing_yds'] / \
    rushing_df['games_played']

rushing_df['capped_yards_per_carry'] = \
    rushing_df['capped_rushing_yds'] / \
    rushing_df['carries']

rushing_df['tds_per_carry'] = \
    rushing_df['rush_touchdown'] / \
    rushing_df['carries']

rushing_df['tds_per_game'] = \
    rushing_df['rush_touchdown'] / \
    rushing_df['games_played']

rushing_df['fumbles_per_carry'] = \
    rushing_df['fumble'] / \
    rushing_df['carries']

rushing_df['fumbles_per_game'] = \
    rushing_df['fumble'] / \
    rushing_df['games_played']

rushing_df = rushing_df[['rusher_player_id',
                         'season',
                         'carries',
                         'games_played',
                         'carries_per_game',
                         'yards_gained',
                         'yards_per_game',
                         'yards_per_carry',
                         'capped_yards_per_game',
                         'capped_yards_per_carry',
                         'rush_touchdown',
                         'tds_per_carry',
                         'tds_per_game',
                         'fumble',
                         'fumbles_per_carry',
                         'fumbles_per_game']]

### RB Workload PCT

In [64]:
rushing_df = rushing_df.merge(roster_map,
                     how='inner',
                     left_on=['rusher_player_id', 'season'],
                     right_on=['player_id', 'season'])

In [65]:
rushing_df =  rushing_df.merge(total_rushes_by_team,
                                 how='left',
                                 left_on=['season', 'team'],
                                 right_on=['season', 'posteam'])

In [66]:
rushing_df['season_workload_pct'] = rushing_df['carries'] / rushing_df['team_carries']
rushing_df['by_game_workload_pct'] = rushing_df['carries_per_game'] / rushing_df['team_carries_per_game']

In [67]:
rushing_df = rushing_df[['rusher_player_id',
                        'season',
                        'team',
                        'games_played',
                        'yards_gained',
                        'carries',
                        'yards_per_game',
                        'yards_per_carry',
                        'capped_yards_per_game',
                        'capped_yards_per_carry',
                        'rush_touchdown',
                        'tds_per_carry',
                        'tds_per_game',
                        'fumble',
                        'fumbles_per_carry',
                        'fumbles_per_game',
                        'carries_per_game',
                        'team_carries',
                        'team_carries_per_game',
                        'season_workload_pct',
                        'by_game_workload_pct']]

## Can we account for roster turnover?

In [68]:
# merge roster_map back on itself with players next season team

roster_map['next_season'] = roster_map['season'] + 1

temp_roster_df = roster_map.merge(roster_map,
                             how='left',
                             left_on=['player_id', 'next_season'],
                             right_on=['player_id', 'season'],
                             suffixes=('', '_x'))

temp_roster_df = temp_roster_df[(temp_roster_df['next_season'] == temp_roster_df['season_x']) |
                                (temp_roster_df['season_x'].isna())]

temp_roster_df = temp_roster_df[['season',
                                 'player_id',
                                 'team',
                                 'team_x']]

temp_roster_df.rename({'team_x':'next_team'}, axis=1, inplace=True)

roster_map = temp_roster_df

In [69]:
rushing_df = rushing_df.merge(roster_map,
                                how='left',
                                left_on=['rusher_player_id', 'season'],
                                right_on=['player_id', 'season'],
                                suffixes=('', '_x'))

#### NOTE: I'm leaving in any players that do not play in the next season during the feature engineering stage, but will remember to remove before training. 

In [70]:
returning_carries_df = rushing_df[rushing_df['team'] == rushing_df['next_team']].\
                                groupby(['team', 'season'])\
                                ['carries'].\
                                sum().\
                                reset_index()

returning_carries_df.columns = ['team', 'season', 'returning_carries']

In [71]:
incoming_carries_df = rushing_df.groupby(['next_team', 'season'])\
                            ['carries'].\
                            sum().\
                            reset_index()

incoming_carries_df.columns = ['team', 'season', 'incoming_carries']

#### NOTE: Both returning_carries and incoming_carries are attemps to evaluate the carryover / changes in talent on a roster from a past season to a future one. This isn't forward leakage under the assumption that you know a team's roster for the upcoming season prior to drafting and that you know each player on that roster's carries for the previous season. 

In [72]:
incoming_carries_df

,team,season,incoming_carries
0,ARI,2009,324
1,ARI,2010,265
2,ARI,2011,300
3,ARI,2012,64
4,ARI,2013,201
...,...,...,...
315,WAS,2014,336
316,WAS,2015,189
317,WAS,2016,292
318,WAS,2017,524


In [73]:
total_rushes_by_team = total_rushes_by_team.merge(returning_carries_df,
                                      how='left',
                                      left_on=['posteam', 'season'],
                                      right_on=['team', 'season']).\
                                merge(incoming_carries_df,
                                      how='left',
                                      left_on=['posteam', 'season'],
                                      right_on=['team', 'season'])

total_rushes_by_team.drop(['team_x', 'team_y'], axis=1, inplace=True)

In [76]:
total_rushes_by_team['returning_carry_pct'] = \
    total_rushes_by_team['returning_carries'] / \
    total_rushes_by_team['team_carries']
    
total_rushes_by_team['incoming_carry_pct'] = \
    total_rushes_by_team['incoming_carries'] / \
    total_rushes_by_team['team_carries']

In [77]:
total_rushes_by_team

,posteam,season,team_carries,team_carries_per_game,returning_carries,incoming_carries,returning_carry_pct,incoming_carry_pct
0,ARI,2009,331,22.066667,317.0,324.0,0.957704,0.978852
1,ARI,2010,300,20.000000,144.0,265.0,0.480000,0.883333
2,ARI,2011,349,23.266667,300.0,300.0,0.859599,0.859599
3,ARI,2012,332,22.133333,3.0,64.0,0.009036,0.192771
4,ARI,2013,382,25.466667,151.0,201.0,0.395288,0.526178
...,...,...,...,...,...,...,...,...
348,WAS,2015,382,25.466667,189.0,189.0,0.494764,0.494764
349,WAS,2016,353,23.533333,246.0,292.0,0.696884,0.827195
350,WAS,2017,376,25.066667,315.0,524.0,0.837766,1.393617
351,WAS,2018,395,26.333333,291.0,382.0,0.736709,0.967089
